In [26]:
#IMPORTANDO BIBLIOTECAS
import requests
import json
import pandas as pd
import sqlite3
import time
from tqdm import tqdm
import datetime
import os

In [27]:
#GUARDANDO LISTAS
Urls_Magalu = []
Sellers_Magalu = []
Country_Magalu = []
Price_Magalu = []
SKU_Magalu = []
Title_Magalu = []
Installment_Magalu_quantidade = []
Installment_Magalu_valor_parcela = []
Installment_Magalu_valor_total = []

In [28]:
def getting_n_creating_magazine_urls(brand):
    # Pegando caminho do database
    Database_path = r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1" + "\Data\\" + brand + "\\" + brand + ".db"

    table = brand + "_products"

    #Criando a Query
    query = "SELECT * FROM " + table

    #Entrando dentro do databse
    connection = sqlite3.connect(Database_path)

    #Criando o dataset em brando
    df = pd.read_sql_query(query, connection)

    #Passando todo o Dataframe para LowerCase
    df = df.apply(lambda x: x.astype(str).str.lower())

    #Arrumando espaços vazios
    # Arrumano os espaços vazios
    df['Product_Name'] = df['Product_Name'].str.replace(" ", "%2B")
    df['Product_Name'] = df['Product_Name'].str.replace("-", "%2B")

    # Criando uma nova coluna no database com a formatação certa
    df['Urls'] = df['Brand'] + "+" + df['Product_Name']

    # Criando a nova coluna que são as urls de pesquisa
    df['Urls_search'] = "https://www.magazineluiza.com.br/_next/data/SJCtTE8TVyZNMypTAB7uE/busca/" + \
    df['Brand'][0] + "%2B" + df['Product_Name'] + ".json?slug=busca&slug=" + \
    df['Brand'][0] + "%2B" + df['Product_Name']

    return df

In [29]:
Df_products_raw = getting_n_creating_magazine_urls("Wacom")
Df_products_raw

,Brand,Product_Name,Product_Descrition,Urls,Urls_search
0,wacom,ctl472,ctl 472 - mesa digitalizadora one by wacom peq...,wacom+ctl472,https://www.magazineluiza.com.br/_next/data/SJ...
1,wacom,ctl4100,ctl4100 - mesa digitalizadora wacom intuos sma...,wacom+ctl4100,https://www.magazineluiza.com.br/_next/data/SJ...
2,wacom,ctl4100wle0,ctl4100wle0 - mesa digitalizadora wacom intuos...,wacom+ctl4100wle0,https://www.magazineluiza.com.br/_next/data/SJ...
3,wacom,ctl4100wlk0,ctl4100wlk0 - mesa digitalizadora wacom intuos...,wacom+ctl4100wlk0,https://www.magazineluiza.com.br/_next/data/SJ...
4,wacom,ctl6100wlk0,ctl6100wlk0 - mesa digitalizadora wacom intuos...,wacom+ctl6100wlk0,https://www.magazineluiza.com.br/_next/data/SJ...
5,wacom,dtc133w0a,dtc133w0a - mesa digitalizadora wacom one crea...,wacom+dtc133w0a,https://www.magazineluiza.com.br/_next/data/SJ...
6,wacom,pth460,pth460 - mesa digitalizadora wacom intuos pro ...,wacom+pth460,https://www.magazineluiza.com.br/_next/data/SJ...
7,wacom,dtk1660k0a,dtk1660k0a - display interativo wacom cintiq 1...,wacom+dtk1660k0a,https://www.magazineluiza.com.br/_next/data/SJ...
8,wacom,pth660,pth660 - mesa digitalizadora wacom intuos pro ...,wacom+pth660,https://www.magazineluiza.com.br/_next/data/SJ...
9,wacom,ctl6100wle0,ctl6100wle0 - mesa digitalizadora wacom intuos...,wacom+ctl6100wle0,https://www.magazineluiza.com.br/_next/data/SJ...


In [30]:
Df_products_raw['Urls_search'][0]

'https://www.magazineluiza.com.br/_next/data/SJCtTE8TVyZNMypTAB7uE/busca/wacom%2Bctl472.json?slug=busca&slug=wacom%2Bctl472'

In [31]:
url = 'https://www.magazineluiza.com.br/_next/data/SJCtTE8TVyZNMypTAB7uE/busca/wacom%2Bctl472.json?slug=busca&slug=wacom%2Bctl472'

In [32]:
headers_magalu = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36 OPR/86.0.4363.59"}

In [33]:
response = requests.get(url, headers=headers_magalu)
texto_json = response.json()

In [34]:
texto_json['pageProps']['data']['search']['products']

[{'id': '2185384',
  'variationId': '218538400',
  'title': 'Mesa Digitalizadora Wacom Pequena',
  'description': '',
  'image': 'https://a-static.mlcdn.com.br/{w}x{h}/mesa-digitalizadora-wacom-pequena-one-ctl472/magazineluiza/218538400/e29858a826bd589161a8d9ccc3686a70.jpg',
  'available': True,
  'position': 1,
  'isBuyBox': False,
  'url': 'mesa-digitalizadora-wacom-pequena-one-ctl472/p/2185384/in/mddi/',
  'path': 'mesa-digitalizadora-wacom-pequena-one-ctl472/p/218538400/in/mddi/',
  'reference': 'One CTL472',
  'attributes': [],
  'seller': {'id': 'magazineluiza',
   'sku': '218538400',
   'description': 'Magalu',
   'category': '1p',
   'deliveryId': 'magazineluiza',
   'deliveryDescription': 'Magalu',
   'details': None,
   '__typename': 'Seller'},
  'brand': {'label': 'Wacom', 'slug': 'wacom', '__typename': 'Brand'},
  'price': {'price': '599.00',
   'fullPrice': '369.99',
   'bestPrice': '351.49',
   'discount': '5.00',
   '__typename': 'Price'},
  'installment': {'paymentMetho

In [35]:
def get_attributes(url):
    time.sleep(20)

    response = requests.get(url, headers=headers_magalu)
    text_json = response.json()

    # URL
    id_url = 0
    for i in text_json['pageProps']['data']['search']['products']:
        try:
            Urls_Magalu.append(text_json['pageProps']['data']['search']['products'][id_url]['url'])
        except:
            Urls_Magalu.append("ERRO")

        id_url = id_url + 1

    # ID
    id_number = 0
    for i in text_json['pageProps']['data']['search']['products']:
        try:
            SKU_Magalu.append(text_json['pageProps']['data']['search']['products'][id_number]['variationId'])
        except:
            SKU_Magalu.append("ERRO")

        id_number = id_number + 1

    # SELLER
    id_seller = 0
    for i in text_json['pageProps']['data']['search']['products']:
        try:
            Sellers_Magalu.append(
                text_json['pageProps']['data']['search']['products'][id_seller]['seller']['description'])
        except:
            Sellers_Magalu.append("ERRO")

        id_seller = id_seller + 1

    # COUNTRY
    id_country = 0
    for i in text_json['pageProps']['data']['search']['products']:
        try:
            Country_Magalu.append(
                text_json['pageProps']['data']['search']['products'][id_country]['seller']['details']['address'][
                    'country'])
        except:
            Country_Magalu.append("ERRO")

        id_country = id_country + 1

    # PREÇO
    id_price = 0
    for i in text_json['pageProps']['data']['search']['products']:
        try:
            Price_Magalu.append(text_json['pageProps']['data']['search']['products'][id_price]['price']['bestPrice'])
        except:
            Price_Magalu.append("ERRO")

        id_price = id_price + 1

    # TITULO
    id_title = 0
    for i in text_json['pageProps']['data']['search']['products']:
        try:
            Title_Magalu.append(text_json['pageProps']['data']['search']['products'][id_title]['title'])
        except:
            Title_Magalu.append("ERRO")

        id_title = id_title + 1

    # INSTALLMENT QUANTIDADE
    id_installment_quantidade = 0
    for i in text_json['pageProps']['data']['search']['products']:
        try:
            Installment_Magalu_quantidade.append(
                text_json['pageProps']['data']['search']['products'][id_installment_quantidade]['installment'][
                    'quantity'])
        except:
            Installment_Magalu_quantidade.append("ERRO")

        id_installment_quantidade = id_installment_quantidade + 1

    # INSTALLMENT VALOR POR PARCELA
    id_installment_valor_parcela = 0
    for i in text_json['pageProps']['data']['search']['products']:
        try:
            Installment_Magalu_valor_parcela.append(
                text_json['pageProps']['data']['search']['products'][id_installment_valor_parcela]['installment'][
                    'amount'])
        except:
            Installment_Magalu_valor_parcela.append("ERRO")

        id_installment_valor_parcela = id_installment_valor_parcela + 1

    # INSTALLMENT VALOR TOTAL
    id_installment_valor_total = 0
    for i in text_json['pageProps']['data']['search']['products']:
        try:
            Installment_Magalu_valor_total.append(
                text_json['pageProps']['data']['search']['products'][id_installment_valor_total]['installment'][
                    'totalAmount'])
        except:
            Installment_Magalu_valor_total.append("ERRO")

        id_installment_valor_total = id_installment_valor_total + 1

In [36]:
for url in tqdm(Df_products_raw['Urls_search']):
    get_attributes(url)

100%|██████████| 10/10 [03:28<00:00, 20.90s/it]


In [68]:
def creating_dataframe(urls, sellers, price, installment, parcel, installment_total, sku, title, brand):
    Dataframe = pd.DataFrame()

    Hoje = pd.to_datetime('today', errors='ignore').date()

    Dataframe['Urls_2'] = urls

    Dataframe['DATE'] = Hoje

    Dataframe['URL'] = "https://www.magazineluiza.com.br/" + Dataframe['Urls_2']
    Dataframe['MARKETPLACE'] = 'MAGAZINE LUIZA'
    Dataframe['SELLER'] = sellers

    Dataframe['PRICE'] = price
    Dataframe['PRICE'] = Dataframe['PRICE'].astype('float')


    Dataframe['INSTALLMENT'] = installment

    Dataframe['PARCEL'] = parcel


    Dataframe['INSTALLMENT_PAYMENT'] = installment_total
    #Dataframe['INSTALLMENT_PAYMENT'] = Dataframe['INSTALLMENT_PAYMENT'].astype('float')

    Dataframe['ID'] = sku
    Dataframe['PRODUCT'] = title

    if brand == "GoPro":
        Dataframe = Dataframe[Dataframe['PRICE'] > 1000]
    elif brand == 'Motorola':
        Dataframe = Dataframe[Dataframe['PRICE'] > 100]
    elif brand == 'Wacom':
        Dataframe = Dataframe[Dataframe['PRICE'] > 100]

    #Colocando na ordem correta
    Dataframe = Dataframe[['DATE', 'URL', 'MARKETPLACE', 'SELLER', 'PRICE', 'PARCEL', 'INSTALLMENT', 'INSTALLMENT_PAYMENT', 'ID', 'PRODUCT']]

    Dataframe['INSTALLMENT'] = Dataframe['INSTALLMENT'].astype('float')
    Dataframe['PARCEL'] = Dataframe['PARCEL'].astype('int')
    Dataframe['INSTALLMENT_PAYMENT'] = Dataframe['INSTALLMENT_PAYMENT'].astype('float')

    return Dataframe

In [69]:
dataset_magalu = creating_dataframe(Urls_Magalu, Sellers_Magalu, Price_Magalu, Installment_Magalu_valor_parcela, Installment_Magalu_quantidade, Installment_Magalu_valor_total, SKU_Magalu, Title_Magalu, 'Wacom')

In [70]:
dataset_magalu

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT
0,2022-05-26,https://www.magazineluiza.com.br/mesa-digitali...,MAGAZINE LUIZA,Magalu,351.49,3,123.33,369.99,218538400,Mesa Digitalizadora Wacom Pequena
1,2022-05-26,https://www.magazineluiza.com.br/mesa-digitali...,MAGAZINE LUIZA,SHOPAR,206.51,3,70.97,212.90,cdh7fb5df7,Mesa Digitalizadora Wacom One CTL-472 Pequena ...
2,2022-05-26,https://www.magazineluiza.com.br/mesa-digitali...,MAGAZINE LUIZA,KaBuM!,199.90,4,58.80,235.18,jeff7hbg33,Mesa Digitalizadora One by Wacom CTL472 Pequena
3,2022-05-26,https://www.magazineluiza.com.br/mesa-digitali...,MAGAZINE LUIZA,Miranda Computação,288.88,5,62.80,314.00,bb7f1kfbj8,Mesa Digitalizadora One by Wacom CTL472 WACOM
4,2022-05-26,https://www.magazineluiza.com.br/mesa-digitali...,MAGAZINE LUIZA,Primetek,306.81,7,47.13,329.90,cg2ck1b5hk,Mesa Digitalizadora One by Wacom CTL472 Pequena
...,...,...,...,...,...,...,...,...,...,...
137,2022-05-26,https://www.magazineluiza.com.br/mesa-digitali...,MAGAZINE LUIZA,Inpower,1375.20,10,152.80,1528.00,eah018g7h7,Mesa Digitalizadora Wacom Intuos M Bluetooth P...
138,2022-05-26,https://www.magazineluiza.com.br/mesa-digitali...,MAGAZINE LUIZA,W-Tech Imports,1937.09,10,199.70,1997.00,hc31e0gkf5,Mesa Digitalizadora Wacom Intuos CTL6100WLE0 B...
139,2022-05-26,https://www.magazineluiza.com.br/mesa-digitali...,MAGAZINE LUIZA,Miranda Computação,1852.88,10,201.40,2014.00,ha211gfj8c,"Mesa Digitalizadora Intuos Bluetooth, Média, V..."
140,2022-05-26,https://www.magazineluiza.com.br/mesa-digitali...,MAGAZINE LUIZA,KaBuM!,1249.90,10,147.05,1470.47,hj7k4kabej,Mesa Digitalizadora Wacom Média Bluetooth Verd...


In [58]:
dataset_magalu[dataset_magalu['INSTALLMENT'] == 'ERRO']

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT


In [71]:
dataset_magalu.dtypes

DATE                    object
URL                     object
MARKETPLACE             object
SELLER                  object
PRICE                  float64
PARCEL                   int32
INSTALLMENT            float64
INSTALLMENT_PAYMENT    float64
ID                      object
PRODUCT                 object
dtype: object

In [40]:
current = os.getcwd()
current

'C:\\Users\\pedro\\Documents\\Turte Brand Protection\\Turtle_Thinker_Alpha_0.1\\Testes_Spiders'

In [41]:
path = current + '\Teste_Magalu.xlsx'

In [21]:
path

'c:\\Users\\pedro\\Documents\\Turte Brand Protection\\Turtle_Thinker_Alpha_0.1\\Testes_Spiders\\Teste_Magalu.xlsx'

In [73]:
dataset_magalu.to_excel(path, index=False)

In [16]:
dataset_magalu.dtypes

DATE                    object
URL                     object
MARKETPLACE             object
SELLER                  object
PRICE                  float64
PARCEL                  object
INSTALLMENT             object
INSTALLMENT_PAYMENT     object
ID                      object
PRODUCT                 object
dtype: object